In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/03/03 16:07:45 WARN Utils: Your hostname, HPU202105030 resolves to a loopback address: 127.0.1.1; using 172.28.74.113 instead (on interface eth0)
23/03/03 16:07:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 16:07:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

--2023-03-01 21:46:10--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.64.100, 54.231.170.41, 52.217.173.249, ...
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.64.100|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-03-01 21:46:11 ERROR 403: Forbidden.



In [ ]:
!gzip -d fhvhv_tripdata_2021-01.csv.gz
!wc -l fhvhv_tripdata_2021-01.csv

In [9]:
df = spark.read.option("header", "true").csv("fhvhv_tripdata_2021-01.csv")
df.schema

In [11]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [15]:
df_pandas = pd.read_csv('head.csv')
df_pandas.dtypes

In [17]:
spark.createDataFrame(df_pandas).schema

/home/gerald/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/gerald/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [19]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [20]:
df = spark.read.option("header", "true").schema(schema).csv('fhvhv_tripdata_2021-01.csv')

In [22]:
df = df.repartition(24)

In [24]:
df.write.parquet('fhvhv/2021/01/')

23/03/01 22:15:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/03/01 22:15:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/03/01 22:15:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/03/01 22:15:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [25]:
df = spark.read.parquet('fhvhv/2021/01/')

In [28]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [29]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').filter(df.hvfhs_license_num=='HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 14:19:18|2021-01-01 14:51:27|         229|         132|
|2021-01-02 21:42:11|2021-01-02 21:52:29|         231|         246|
|2021-01-01 02:59:54|2021-01-01 03:13:00|          85|          35|
|2021-01-01 00:35:57|2021-01-01 01:01:29|          40|         129|
|2021-01-01 01:59:32|2021-01-01 02:11:47|         263|           7|
|2021-01-01 01:45:04|2021-01-01 01:51:49|          37|          36|
|2021-01-01 03:41:19|2021-01-01 03:49:23|         232|         148|
|2021-01-01 02:36:17|2021-01-01 02:44:45|         167|         168|
|2021-01-03 15:18:05|2021-01-03 15:39:45|         188|          37|
|2021-01-04 07:17:45|2021-01-04 07:22:03|         216|         216|
|2021-01-01 19:28:55|2021-01-01 19:34:47|          36|          36|
|2021-01-01 00:30:05|2021-01-01 00:40:31|       